In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

In [65]:
pd.options.display.expand_frame_repr = False
pd.set_option('display.max_columns', None)

### Donload data + join train and test
Join required for OHE to avoid loosing features in test

In [66]:
train = pd.read_csv('train.csv', index_col='Id')
test = pd.read_csv('test.csv', index_col='Id')
y_train = np.log1p(train['SalePrice'])
train.drop(['SalePrice'], axis=1, inplace=True)
X_combined = train.append(test)
X_combined

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,7,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Gd,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,252.0,Unf,0.0,294.0,546.0,GasA,TA,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,CarPort,1970.0,Unf,1.0,286.0,TA,TA,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,5,7,1960,1996,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,1224.0,Unf,0.0,0.0,1224.0,GasA,Ex,Y,SBrkr,1224,0,0,1224,1.0,0.0,1,0,4,1,TA,7,Typ,1,TA,Detchd,1960.0,Unf,2.0,576.0,TA,TA,Y,474,0,0,0,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml


#### Preprocessing and coding

In [67]:
def checkLosses(data, target=None):
    '''
    функция подсчитывает пропуски и оценивает их значимость для определения классов
    '''
    
    # подсчет кол-ва и процента пропусков
    losses = data.isnull().sum()
    losses = pd.DataFrame(losses[losses>0], columns=['loss'])
    losses = losses.sort_values('loss', ascending=False)
    losses['loss%'] = losses['loss']/len(data)*100
    losses['loss%'] = losses['loss%'].round(1)
    
    if target:
        for col in losses.index:

            # подсчет разницы вероятности класса для данных с пропусками и без них 
            table1 = data.loc[data[col].isnull(), target].value_counts(normalize=True)
            table2 = data.loc[data[col].notnull(), target].value_counts(normalize=True)
            deltas = (abs(table1 - table2)*100).round(1)
            for i in deltas.index:
                losses.loc[col,'delta_'+str(i)] = deltas.loc[i]        

            # интегральная оценка важности пропуска
            losses.loc[col,'importance'] = np.round(deltas.sum()/2, 1)

            # кол-во уникальных значений (для определения категориальных признаков)
            losses.loc[col,'unique'] = data[col].nunique()
        
    return losses

In [69]:
def fill_losses(X_combined):
    
    # Alley : data description says NA means "no alley access"
    X_combined.loc[:, "Alley"] = X_combined.loc[:, "Alley"].fillna("None")
    # BedroomAbvGr : NA most likely means 0
    train.loc[:, "BedroomAbvGr"] = X_combined.loc[:, "BedroomAbvGr"].fillna(0)
    # BsmtQual etc : data description says NA for basement features is "no basement"
    X_combined.loc[:, "BsmtQual"] = X_combined.loc[:, "BsmtQual"].fillna("No")
    X_combined.loc[:, "BsmtCond"] = X_combined.loc[:, "BsmtCond"].fillna("No")
    X_combined.loc[:, "BsmtExposure"] = X_combined.loc[:, "BsmtExposure"].fillna("No")
    X_combined.loc[:, "BsmtFinType1"] = X_combined.loc[:, "BsmtFinType1"].fillna("No")
    X_combined.loc[:, "BsmtFinType2"] = X_combined.loc[:, "BsmtFinType2"].fillna("No")
    X_combined.loc[:, "BsmtFullBath"] = X_combined.loc[:, "BsmtFullBath"].fillna(0)
    X_combined.loc[:, "BsmtHalfBath"] = X_combined.loc[:, "BsmtHalfBath"].fillna(0)
    X_combined.loc[:, "BsmtUnfSF"] = X_combined.loc[:, "BsmtUnfSF"].fillna(0)
    # CentralAir : NA most likely means No
    X_combined.loc[:, "CentralAir"] = X_combined.loc[:, "CentralAir"].fillna("N")
    # Condition : NA most likely means Normal
    X_combined.loc[:, "Condition1"] = X_combined.loc[:, "Condition1"].fillna("Norm")
    X_combined.loc[:, "Condition2"] = X_combined.loc[:, "Condition2"].fillna("Norm")
    # EnclosedPorch : NA most likely means no enclosed porch
    X_combined.loc[:, "EnclosedPorch"] = X_combined.loc[:, "EnclosedPorch"].fillna(0)
    # External stuff : NA most likely means average
    X_combined.loc[:, "ExterCond"] = X_combined.loc[:, "ExterCond"].fillna("TA")
    X_combined.loc[:, "ExterQual"] = X_combined.loc[:, "ExterQual"].fillna("TA")
    # Fence : data description says NA means "no fence"
    X_combined.loc[:, "Fence"] = X_combined.loc[:, "Fence"].fillna("No")
    # FireplaceQu : data description says NA means "no fireplace"
    X_combined.loc[:, "FireplaceQu"] = X_combined.loc[:, "FireplaceQu"].fillna("No")
    X_combined.loc[:, "Fireplaces"] = X_combined.loc[:, "Fireplaces"].fillna(0)
    # Functional : data description says NA means typical
    X_combined.loc[:, "Functional"] = X_combined.loc[:, "Functional"].fillna("Typ")
    # GarageType etc : data description says NA for garage features is "no garage"
    X_combined.loc[:, "GarageType"] = X_combined.loc[:, "GarageType"].fillna("No")
    X_combined.loc[:, "GarageFinish"] = X_combined.loc[:, "GarageFinish"].fillna("No")
    X_combined.loc[:, "GarageQual"] = X_combined.loc[:, "GarageQual"].fillna("No")
    X_combined.loc[:, "GarageCond"] = X_combined.loc[:, "GarageCond"].fillna("No")
    X_combined.loc[:, "GarageArea"] = X_combined.loc[:, "GarageArea"].fillna(0)
    X_combined.loc[:, "GarageCars"] = X_combined.loc[:, "GarageCars"].fillna(0)
    # HalfBath : NA most likely means no half baths above grade
    X_combined.loc[:, "HalfBath"] = X_combined.loc[:, "HalfBath"].fillna(0)
    # HeatingQC : NA most likely means typical
    X_combined.loc[:, "HeatingQC"] = X_combined.loc[:, "HeatingQC"].fillna("TA")
    # KitchenAbvGr : NA most likely means 0
    X_combined.loc[:, "KitchenAbvGr"] = X_combined.loc[:, "KitchenAbvGr"].fillna(0)
    # KitchenQual : NA most likely means typical
    X_combined.loc[:, "KitchenQual"] = X_combined.loc[:, "KitchenQual"].fillna("TA")
    # LotFrontage : NA most likely means no lot frontage
    X_combined.loc[:, "LotFrontage"] = X_combined.loc[:, "LotFrontage"].fillna(0)
    # LotShape : NA most likely means regular
    X_combined.loc[:, "LotShape"] = X_combined.loc[:, "LotShape"].fillna("Reg")
    # MasVnrType : NA most likely means no veneer
    X_combined.loc[:, "MasVnrType"] = X_combined.loc[:, "MasVnrType"].fillna("None")
    X_combined.loc[:, "MasVnrArea"] = X_combined.loc[:, "MasVnrArea"].fillna(0)
    # MiscFeature : data description says NA means "no misc feature"
    X_combined.loc[:, "MiscFeature"] = X_combined.loc[:, "MiscFeature"].fillna("No")
    X_combined.loc[:, "MiscVal"] = X_combined.loc[:, "MiscVal"].fillna(0)
    # OpenPorchSF : NA most likely means no open porch
    X_combined.loc[:, "OpenPorchSF"] = X_combined.loc[:, "OpenPorchSF"].fillna(0)
    # PavedDrive : NA most likely means not paved
    X_combined.loc[:, "PavedDrive"] = X_combined.loc[:, "PavedDrive"].fillna("N")
    # PoolQC : data description says NA means "no pool"
    X_combined.loc[:, "PoolQC"] = X_combined.loc[:, "PoolQC"].fillna("No")
    X_combined.loc[:, "PoolArea"] = X_combined.loc[:, "PoolArea"].fillna(0)
    # SaleCondition : NA most likely means normal sale
    X_combined.loc[:, "SaleCondition"] = X_combined.loc[:, "SaleCondition"].fillna("Normal")
    # ScreenPorch : NA most likely means no screen porch
    X_combined.loc[:, "ScreenPorch"] = X_combined.loc[:, "ScreenPorch"].fillna(0)
    # TotRmsAbvGrd : NA most likely means 0
    X_combined.loc[:, "TotRmsAbvGrd"] = X_combined.loc[:, "TotRmsAbvGrd"].fillna(0)
    # Utilities : NA most likely means all public utilities
    X_combined.loc[:, "Utilities"] = X_combined.loc[:, "Utilities"].fillna("AllPub")
    # WoodDeckSF : NA most likely means no wood deck
    X_combined.loc[:, "WoodDeckSF"] = X_combined.loc[:, "WoodDeckSF"].fillna(0)
    
    return X_combined

In [70]:
def change_categorical_numeric(X_combined):
    # Some numerical features are actually really categories
    X_combined = X_combined.replace({
    "MSSubClass" : {20 : "SC20", 30 : "SC30", 40 : "SC40", 45 : "SC45", 
                     50 : "SC50", 60 : "SC60", 70 : "SC70", 75 : "SC75", 
                     80 : "SC80", 85 : "SC85", 90 : "SC90", 120 : "SC120", 
                     150 : "SC150", 160 : "SC160", 180 : "SC180", 190 : "SC190"},
    "MoSold" : {1 : "Jan", 2 : "Feb", 3 : "Mar", 4 : "Apr", 5 : "May", 6 : "Jun",
                7 : "Jul", 8 : "Aug", 9 : "Sep", 10 : "Oct", 11 : "Nov", 12 : "Dec"}
    })
    
    # Encode some categorical features as ordered numbers when there is information in the order
    X_combined = X_combined.replace({
        #"Alley" : {"Grvl" : 1, "Pave" : 2},
        "BsmtCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "BsmtExposure" : {"No" : 0, "Mn" : 1, "Av": 2, "Gd" : 3},
        "BsmtFinType1" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
        "BsmtFinType2" : {"No" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
        "BsmtQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5},
        "ExterCond" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
        "ExterQual" : {"Po" : 1, "Fa" : 2, "TA": 3, "Gd": 4, "Ex" : 5},
        "FireplaceQu" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "Functional" : {"Sal" : 1, "Sev" : 2, "Maj2" : 3, "Maj1" : 4, "Mod": 5, "Min2" : 6, "Min1" : 7, "Typ" : 8},
        "GarageCond" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "GarageQual" : {"No" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "HeatingQC" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "KitchenQual" : {"Po" : 1, "Fa" : 2, "TA" : 3, "Gd" : 4, "Ex" : 5},
        "LandSlope" : {"Sev" : 1, "Mod" : 2, "Gtl" : 3},
        "LotShape" : {"IR3" : 1, "IR2" : 2, "IR1" : 3, "Reg" : 4},
        "PavedDrive" : {"N" : 0, "P" : 1, "Y" : 2},
        "PoolQC" : {"No" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, "Ex" : 4},
        "Street" : {"Grvl" : 1, "Pave" : 2},
        "Utilities" : {"ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, "AllPub" : 4}
    })
    
    return X_combined

In [71]:
def create_polinomial_features(X_combined):
# Create new features
    # 3* Polynomials on the top 10 existing features
    X_combined["OverallQual-s2"] = X_combined["OverallQual"] ** 2
    X_combined["OverallQual-s3"] = X_combined["OverallQual"] ** 3
    X_combined["OverallQual-Sq"] = np.sqrt(X_combined["OverallQual"])
    X_combined["AllSF-2"] = X_combined["AllSF"] ** 2
    X_combined["AllSF-3"] = X_combined["AllSF"] ** 3
    X_combined["AllSF-Sq"] = np.sqrt(X_combined["AllSF"])
    X_combined["AllFlrsSF-2"] = X_combined["AllFlrsSF"] ** 2
    X_combined["AllFlrsSF-3"] = X_combined["AllFlrsSF"] ** 3
    X_combined["AllFlrsSF-Sq"] = np.sqrt(X_combined["AllFlrsSF"])
    X_combined["GrLivArea-2"] = X_combined["GrLivArea"] ** 2
    X_combined["GrLivArea-3"] = X_combined["GrLivArea"] ** 3
    X_combined["GrLivArea-Sq"] = np.sqrt(X_combined["GrLivArea"])
    X_combined["SimplOverallQual-s2"] = X_combined["SimplOverallQual"] ** 2
    X_combined["SimplOverallQual-s3"] = X_combined["SimplOverallQual"] ** 3
    X_combined["SimplOverallQual-Sq"] = np.sqrt(X_combined["SimplOverallQual"])
    X_combined["ExterQual-2"] = X_combined["ExterQual"] ** 2
    X_combined["ExterQual-3"] = X_combined["ExterQual"] ** 3
    X_combined["ExterQual-Sq"] = np.sqrt(X_combined["ExterQual"])
    X_combined["GarageCars-2"] = X_combined["GarageCars"] ** 2
    X_combined["GarageCars-3"] = X_combined["GarageCars"] ** 3
    X_combined["GarageCars-Sq"] = np.sqrt(X_combined["GarageCars"])
    X_combined["TotalBath-2"] = X_combined["TotalBath"] ** 2
    X_combined["TotalBath-3"] = X_combined["TotalBath"] ** 3
    X_combined["TotalBath-Sq"] = np.sqrt(X_combined["TotalBath"])
    X_combined["KitchenQual-2"] = X_combined["KitchenQual"] ** 2
    X_combined["KitchenQual-3"] = X_combined["KitchenQual"] ** 3
    X_combined["KitchenQual-Sq"] = np.sqrt(X_combined["KitchenQual"])
    X_combined["GarageScore-2"] = X_combined["GarageScore"] ** 2
    X_combined["GarageScore-3"] = X_combined["GarageScore"] ** 3
    X_combined["GarageScore-Sq"] = np.sqrt(X_combined["GarageScore"])
    return X_combined

In [72]:
X_combined = fill_losses(X_combined)
X_combined = change_categorical_numeric(X_combined)
    
X_combined.drop('GarageYrBlt', axis=1, inplace=True)

for col in ['MSZoning','Exterior1st','Exterior2nd','Electrical','SaleType']:
    X_combined.loc[X_combined[col].isna(),col] = X_combined[col].value_counts().index[0]

for col in ['BsmtFinSF1','BsmtFinSF2','TotalBsmtSF']:
    X_combined.loc[X_combined[col].isna(),col] = X_combined[col].median()

In [73]:
cat_cols = list(X_combined.select_dtypes(include = ["object"]).columns)
num_cols = list(X_combined.select_dtypes(exclude = ["object"]).columns)
len(cat_cols), len(num_cols)

(26, 52)

In [74]:
# убираем скошенность
'''from scipy.stats import skew
skewed_feats = X_combined[num_cols].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[abs(skewed_feats) > 0.75].index
X_combined[skewed_feats] = np.log1p(X_combined[skewed_feats])'''

'from scipy.stats import skew\nskewed_feats = X_combined[num_cols].apply(lambda x: skew(x.dropna())) #compute skewness\nskewed_feats = skewed_feats[abs(skewed_feats) > 0.75].index\nX_combined[skewed_feats] = np.log1p(X_combined[skewed_feats])'

In [75]:
# удалить оставшиеся столбцы с пропусками (если такие еще есть)
X_combined.dropna(axis=1, inplace=True)
X_combined.shape

(2919, 78)

In [76]:
X_combined_ohe = pd.get_dummies(X_combined, drop_first=True)
#scaler = StandardScaler()
#X_combined_ohe[num_cols] = scaler.fit_transform(X_combined_ohe[num_cols])
X_combined_ohe

,LotFrontage,LotArea,Street,LotShape,Utilities,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,YrSold,MSSubClass_SC150,MSSubClass_SC160,MSSubClass_SC180,MSSubClass_SC190,MSSubClass_SC20,MSSubClass_SC30,MSSubClass_SC40,MSSubClass_SC45,MSSubClass_SC50,MSSubClass_SC60,MSSubClass_SC70,MSSubClass_SC75,MSSubClass_SC80,MSSubClass_SC85,MSSubClass_SC90,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Alley_None,Alley_Pave,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_No,GarageFinish_No,GarageFinish_RFn,GarageFinish_Unf,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_No,MiscFeature_No,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_Aug,MoSold_Dec,MoSold_Feb,MoSold_Jan,MoSold_Jul,MoSold_Jun,MoSold_Mar,MoSold_May,MoSold_Nov,MoSold_Oct,MoSold_Sep,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450,2,4,4,3,7,5,2003,2003,196.0,4,3,4,3,0,6,706.0,1,0.0,150.0,856.0,5,856,854,0,1710,1.0,0.0,2,1,3,1,4,8,8,0,0,2.0,548.0,3,3,2,0,61,0,0,0,0,0,0,2008,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,

In [77]:
encoder = OrdinalEncoder(dtype='int32')
encoder.fit(X_combined[cat_cols])
X_combined_ord = X_combined.copy()
X_combined_ord[cat_cols] = pd.DataFrame(encoder.transform(X_combined[cat_cols]), columns=cat_cols, index=X_combined.index)
X_combined_ord

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,10,3,65.0,8450,2,1,4,3,4,4,3,5,2,2,0,5,7,5,2003,2003,1,1,12,13,1,196.0,4,3,2,4,3,0,6,706.0,1,0.0,150.0,856.0,1,5,1,4,856,854,0,1710,1.0,0.0,2,1,3,1,4,8,8,0,0,1,2,2.0,548.0,3,3,2,0,61,0,0,0,0,0,4,1,0,3,2008,8,4
2,5,3,80.0,9600,2,1,4,3,4,2,3,24,1,2,0,2,6,8,1976,1976,1,1,8,8,2,0.0,3,3,1,4,3,3,5,978.0,1,0.0,284.0,1262.0,1,5,1,4,1262,0,0,1262,0.0,1.0,2,0,3,1,3,6,8,1,3,1,2,2.0,460.0,3,3,2,298,0,0,0,0,0,0,4,1,0,8,2007,8,4
3,10,3,68.0,11250,2,1,3,3,4,4,3,5,2,2,0,5,7,5,2001,2002,1,1,12,13,1,162.0,4,3,2,4,3,1,6,486.0,1,0.0,434.0,920.0,1,5,1,4,920,866,0,1786,1.0,0.0,2,1,3,1,4,6,8,1,3,1,2,2.0,608.0,3,3,2,0,42,0,0,0,0,0,4,1,0,11,2008,8,4
4,11,3,60.0,9550,2,1,3,3,4,0,3,6,2,2,0,5,7,5,1915,1970,1,1,13,15,2,0.0,3,3,0,3,4,0,5,216.0,1,0.0,540.0,756.0,1,4,1,4,961,756,0,1717,1.0,0.0,1,0,3,1,4,7,8,1,4,5,3,3.0,642.0,3,3,2,0,35,272,0,0,0,0,4,1,0,3,2006,8,0
5,10,3,84.0,14260,2,1,3,3,4,2,3,15,2,2,0,5,8,5,2000,2000,1,1,12,13,1,350.0,4,3,2,4,3,2,6,655.0,1,0.0,490.0,1145.0,1,5,1,4,1145,1053,0,2198,1.0,0.0,2,1,4,1,4,9,8,1,3,1,2,3.0,836.0,3,3,2,192,84,0,0,0,0,0,4,1,0,2,2008,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,2,4,21.0,1936,2,1,4,3,4,4,3,10,2,2,3,5,4,7,1970,1970,1,1,5,5,2,0.0,3,3,1,3,3,0,1,0.0,1,0.0,546.0,546.0,1,4,1,4,546,546,0,1092,0.0,0.0,1,1,3,1,3,5,8,0,0,6,1,0.0,0.0,0,0,2,0,0,0,0,0,0,0,4,1,0,6,2006,8,4
2916,2,4,21.0,1894,2,1,4,3,4,4,3,10,2,2,4,5,4,5,1970,1970,1,1,5,5,2,0.0,3,3,1,3,3,0,3,252.0,1,0.0,294.0,546.0,1,3,1,4,546,546,0,1092,0.0,0.0,1,1,3,1,3,6,8,0,0,4,3,1.0,286.0,3,3,2,0,24,0,0,0,0,0,4,1,0,0,2006,8,0
2917,5,3,160.0,20000,2,1,4,3,4,4,3,11,2,2,0,2,5,7,1960,1996,1,1,12,13,2,0.0,3,3,1,3,3,0,5,1224.0,1,0.0,0.0,1224.0,1,5,1,4,1224,0,0,1224,1.0,0.0,1,0,4,1,3,7,8,1,3,5,3,2.0,576.0,3,3,2,474,0,0,0,0,0,0,4,1,0,11,2006,8,0


split to train and test

In [78]:
X_train_ohe = X_combined_ohe.loc[train.index]
X_test_ohe = X_combined_ohe.loc[test.index]
X_train_ohe.shape, X_test_ohe.shape

((1460, 230), (1459, 230))

In [79]:
X_train_ord = X_combined_ord.loc[train.index]
X_test_ord = X_combined_ord.loc[test.index]
X_train_ord.shape, X_test_ord.shape

((1460, 78), (1459, 78))

In [80]:
X_train_ohe

,LotFrontage,LotArea,Street,LotShape,Utilities,LandSlope,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,YrSold,MSSubClass_SC150,MSSubClass_SC160,MSSubClass_SC180,MSSubClass_SC190,MSSubClass_SC20,MSSubClass_SC30,MSSubClass_SC40,MSSubClass_SC45,MSSubClass_SC50,MSSubClass_SC60,MSSubClass_SC70,MSSubClass_SC75,MSSubClass_SC80,MSSubClass_SC85,MSSubClass_SC90,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Alley_None,Alley_Pave,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_No,GarageFinish_No,GarageFinish_RFn,GarageFinish_Unf,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_No,MiscFeature_No,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_Aug,MoSold_Dec,MoSold_Feb,MoSold_Jan,MoSold_Jul,MoSold_Jun,MoSold_Mar,MoSold_May,MoSold_Nov,MoSold_Oct,MoSold_Sep,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450,2,4,4,3,7,5,2003,2003,196.0,4,3,4,3,0,6,706.0,1,0.0,150.0,856.0,5,856,854,0,1710,1.0,0.0,2,1,3,1,4,8,8,0,0,2.0,548.0,3,3,2,0,61,0,0,0,0,0,0,2008,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,

In [81]:
X_train_ord

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,10,3,65.0,8450,2,1,4,3,4,4,3,5,2,2,0,5,7,5,2003,2003,1,1,12,13,1,196.0,4,3,2,4,3,0,6,706.0,1,0.0,150.0,856.0,1,5,1,4,856,854,0,1710,1.0,0.0,2,1,3,1,4,8,8,0,0,1,2,2.0,548.0,3,3,2,0,61,0,0,0,0,0,4,1,0,3,2008,8,4
2,5,3,80.0,9600,2,1,4,3,4,2,3,24,1,2,0,2,6,8,1976,1976,1,1,8,8,2,0.0,3,3,1,4,3,3,5,978.0,1,0.0,284.0,1262.0,1,5,1,4,1262,0,0,1262,0.0,1.0,2,0,3,1,3,6,8,1,3,1,2,2.0,460.0,3,3,2,298,0,0,0,0,0,0,4,1,0,8,2007,8,4
3,10,3,68.0,11250,2,1,3,3,4,4,3,5,2,2,0,5,7,5,2001,2002,1,1,12,13,1,162.0,4,3,2,4,3,1,6,486.0,1,0.0,434.0,920.0,1,5,1,4,920,866,0,1786,1.0,0.0,2,1,3,1,4,6,8,1,3,1,2,2.0,608.0,3,3,2,0,42,0,0,0,0,0,4,1,0,11,2008,8,4
4,11,3,60.0,9550,2,1,3,3,4,0,3,6,2,2,0,5,7,5,1915,1970,1,1,13,15,2,0.0,3,3,0,3,4,0,5,216.0,1,0.0,540.0,756.0,1,4,1,4,961,756,0,1717,1.0,0.0,1,0,3,1,4,7,8,1,4,5,3,3.0,642.0,3,3,2,0,35,272,0,0,0,0,4,1,0,3,2006,8,0
5,10,3,84.0,14260,2,1,3,3,4,2,3,15,2,2,0,5,8,5,2000,2000,1,1,12,13,1,350.0,4,3,2,4,3,2,6,655.0,1,0.0,490.0,1145.0,1,5,1,4,1145,1053,0,2198,1.0,0.0,2,1,4,1,4,9,8,1,3,1,2,3.0,836.0,3,3,2,192,84,0,0,0,0,0,4,1,0,2,2008,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,10,3,62.0,7917,2,1,4,3,4,4,3,8,2,2,0,5,6,5,1999,2000,1,1,12,13,2,0.0,3,3,2,4,3,0,1,0.0,1,0.0,953.0,953.0,1,5,1,4,953,694,0,1647,0.0,0.0,2,1,3,1,3,7,8,1,3,1,2,2.0,460.0,3,3,2,0,40,0,0,0,0,0,4,1,0,1,2007,8,4
1457,5,3,85.0,13175,2,1,4,3,4,4,3,14,2,2,0,2,6,6,1978,1988,1,1,9,10,3,119.0,3,3,1,4,3,0,5,790.0,3,163.0,589.0,1542.0,1,3,1,4,2073,0,0,2073,1.0,0.0,2,0,3,1,3,7,7,2,3,1,3,2.0,500.0,3,3,2,349,0,0,0,0,0,0,2,1,0,3,2010,8,4
1458,11,3,66.0,9042,2,1,4,3,4,4,3,6,2,2,0,5,7,9,1941,2006,1,1,5,5,2,0.0,5,4,4,3,4,0,6,275.0,1,0.0,877.0,1152.0,1,5,1,4,1188,1152,0,2340,0.0,0.0,2,0,4,1,4,9,8,2,4,1,2,1.0,252.0,3,3,2,0,60,0,0,0,0,0,0,3,2500,8,2010,8,4


#### Save

In [84]:
train_ohe = X_train_ohe.join(y_train)
train_ohe.to_csv('train_ohe.csv')
X_test_ohe.to_csv('test_ohe.csv')

In [85]:
train_ord = X_train_ord.join(y_train)

cols = X_train_ord.columns.values
for i in range(len(cols)):
    if cols[i] in cat_cols:
        cols[i] = 'cat_' + cols[i]
X_train_ord.columns

train_ord.to_csv('train_ord.csv')
X_test_ord.to_csv('test_ord.csv')